In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [5]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [177]:
class DecisionNodeNumerical():
    def __init__(self, feature_name = None, threshold = None, left = None, right = None, info_gain = None):
        self.feature_name = feature_name
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
    def set_info_gain(self, info_gain):
        self.info_gain = info_gain

        
    def set_left(self, left):
        self.left = left
        
    def set_right(self, right):
        self.right = right
        
    def left_query(self):
        return f'`{self.feature_name}` <= {self.threshold}'
    
    def right_query(self):
        return f'`{self.feature_name}` > {self.threshold}'
        
class LeafNode():
    def __init__(self, value, entropy = None, gini = None):
        self.value = value
        self.entropy = entropy
        self.gini = gini
    

class DecisionTreeClassifier:
    
    def __init__(self, max_depth = None, min_sample_leaf = None):
        self.depth = 0
        self.max_depth = max_depth
        self.min_sample_leaf = min_sample_leaf
        
        self.root = None
        
        
    def split(self, df, decisionNode):
        df_left = df.query(decisionNode.left_query())
        df_right = df.query(decisionNode.right_query())
        return (df_left, df_right)
        
    def get_information_gain(self, parent, l_child, r_child, target_col, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.get_gini(parent, target_col) - (weight_l*self.get_gini(l_child, target_col) + weight_r*self.get_gini(r_child, target_col))
        else:
            gain = self.get_entropy(parent, target_col) - (weight_l*self.get_entropy(l_child, target_col) + weight_r*self.get_entropy(r_child, target_col))
        return gain
        
    def get_entropy(self, df, target_col):
        entropy = 0
        for target in np.unique(df[target_col]):
            fraction = df[target_col].value_counts()[target] / len(df[target_col])
            entropy += -fraction * np.log2(fraction)
            
        return entropy
    
    # Dont use this for multi-class stuff
    def get_gini(self, df, target_col):
        gini = 0
        for target in np.unique(df[target_col]):
            fraction = df[target_col].value_counts()[target] / len(df[target_col])
            gini += fraction ** 2
            
        return gini
    
    def generate_leaf_node(self, df, target_col):
        value = df[target_col].mode()[0]
        entropy = self.get_entropy(df, target_col)
        gini = self.get_gini(df, target_col)
        return LeafNode(value, entropy, gini)
    
    def get_best_split(self, df, target_col):
        
        max_info_gain = float("-inf")
        best_decision = None
        
        for column in df.columns:
            if column == target_col:
                continue
            
            possible_thresholds = np.unique(df[column])
            for threshold in possible_thresholds:
                decisionNode = DecisionNodeNumerical(feature_name=column, threshold=threshold)
                df_left, df_right = self.split(df, decisionNode)
                curr_info_gain = self.get_information_gain(df, df_left, df_right, target_col, "entropy")
#                 print(curr_info_gain)
                if curr_info_gain > max_info_gain:
                    decisionNode.set_info_gain(curr_info_gain)
                    best_decision = decisionNode
                    max_info_gain = curr_info_gain
        
        return best_decision 
    
    def build_tree(self, df, target, current_depth):
        if len(df) >= self.min_sample_leaf and current_depth <= self.max_depth:
            best_split = self.get_best_split(df, target)
            left_df, right_df = self.split(df, best_split)
            if best_split.info_gain > 0:
                left_subtree = self.build_tree(left_df, target, current_depth + 1)
                best_split.set_left(left_subtree)
                
                right_subtree = self.build_tree(right_df, target, current_depth + 1)
                best_split.set_right(right_subtree)
                
                return best_split
        
        leaf_node = self.generate_leaf_node(df, target)
        return leaf_node
            
    def fit(self, df, target):
        self.root = self.build_tree(df, target, 0)
        
    def print_tree(self):
        
        def level_order_traversal(root):
            if not root:
                return []

            queue = [root]
            result = []

            while queue:
                level = []
                for _ in range(len(queue)):
                    node = queue.pop(0)
                    if isinstance(node, DecisionNodeNumerical):
                        level.append(node.left_query())
                        if node.left:
                            queue.append(node.left)
                        if node.right:
                            queue.append(node.right)
                    else: 
                        level.append(node.value)
                result.append(level)

            return result
        
        print(level_order_traversal(self.root))
                
                

In [181]:
d = DecisionTreeClassifier(2, 2)

In [182]:
d.fit(df, "target")

In [183]:
d.print_tree()

[['`petal length (cm)` <= 1.9'], [0, '`petal width (cm)` <= 1.7'], ['`petal length (cm)` <= 4.9', '`petal length (cm)` <= 4.8'], [1, 2, 2, 2]]
